In [ ]:
import os
os.environ['PATH']+=':/usr/local/cuda/bin/' # cuda must be in path
os.environ['CUDA_PATH']='/usr/local/cuda'

import idi.simulation as sim
import numpy as np
from numpy import pi
!hostname
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!where nvcc

In [ ]:
import mkl
mkl.domain_set_num_threads(8,'vml')
mkl.set_num_threads(8)

**Settings**

In [ ]:
Natoms = int(1e5)
Ndet = 2048*2
detz = 9 # in cm
pixelsize = 50 # in um
Nimg = 10
E = 9200  # in ev
rotangles = np.array([0,0,0]) / 180 * pi
cuda = True
r = 5.6 #nm for sphere, A for crystal
repeats=np.array([400,400,500])//10
fwhm=np.array([0.1,0.1,0.2])/10

**Simulation**

In [ ]:
print("preparing")
_detz = detz * 1e4 # in um
k = 2 * pi / (1.24 / E)  # in 1/um
N = Natoms
_r = r * 1e-3 # in um
_a = r * 1e-4

simobject = sim.simobj.fcc(E,N,_a,repeats=repeats,fwhm=fwhm)
simobject.rndPhase=True
simobject.rndPos=False
#c=0.3 #um/fs
#tau=.1 #fs

In [ ]:
r1=sim.cuda.simulate(Nimg,simobject,Ndet,pixelsize,_detz,k)
r1=np.abs(r1)**2

In [ ]:
np.savez_compressed('/reg/data/ana15/xpp/xppx37817/scratch/zimmf/msc/fcc_temp2_1.npz',r=r1,detz=_detz,pixelsize=pixelsize,a=_a,Natoms=Natoms,repeats=repeats,E=E,k=k)

In [ ]:
import idi.reconstruction as recon

In [ ]:
import idi.util as util
r=util.rebin(r1,[1,2,2])

In [ ]:
import mkl
mkl.domain_set_num_threads(8,'fft')

In [ ]:
corr=recon.ft.corr(r,_detz/(2*pixelsize),True)/len(r)

In [ ]:
corr=recon.ft.unwrap(corr)

In [ ]:
norm=recon.ft.unwrap(recon.ft.corr(np.mean(r,0),_detz/(2*pixelsize),True))

In [ ]:
import gc
gc.collect()

In [ ]:
t=corr[0]/norm[0]
t[norm[0]<1e-20]=np.nan

In [ ]:
plt.rcParams['figure.figsize']=20,20
plt.matshow(t-1,vmin=0,vmax=.01)
plt.colorbar()